In [1]:
from __future__ import division
from __future__ import print_function  
import warnings
import pandas as pd
import gsp_support as gsp
import matplotlib.pyplot as plt
import numpy as np
pd.options.plotting.backend = "plotly"
import datetime
output_folder = 'FINAL/daily/'
input_folder = 'FINAL/Disaggregation/stage2_filter/'

In [2]:
total = pd.read_csv(input_folder+'3 min disaggregated result FINAL with removed meters.csv')

In [3]:
total['x_Timestamp'] = pd.to_datetime(total['x_Timestamp'])
total = total.set_index('x_Timestamp')

In [4]:
total = total.loc[~total['meter'].isin(['BR11','BR22','BR51'])]

In [5]:
total['hour'] = total.index.hour

In [6]:
len(total['meter'].unique())

24

In [8]:
total

,gsp,power,current,compressor,final_compressor,ac_power,ac_on,voltage,meter,m-y,hour
x_Timestamp,,,,,,,,,,,
2019-05-09 00:00:00,0.0,3440.0,14.74,0,1,1328.598375,1,233.66,MH24,2019-05,0
2019-05-09 00:03:00,0.0,2440.0,10.20,0,1,2051.372306,1,238.77,MH24,2019-05,0
2019-05-09 00:06:00,0.0,2420.0,10.17,0,1,2031.372306,1,239.08,MH24,2019-05,0
2019-05-09 00:09:00,0.0,2420.0,10.20,0,1,2031.372306,1,238.73,MH24,2019-05,0
2019-05-09 00:12:00,0.0,2380.0,10.00,0,1,1991.372306,1,237.68,MH24,2019-05,0
...,...,...,...,...,...,...,...,...,...,...,...
2020-11-29 23:45:00,0.0,0.0,0.00,0,0,0.000000,0,151.98,MH21,2020-11,23
2020-11-29 23:48:00,0.0,0.0,0.00,0,0,0.000000,0,151.71,MH21,2020-11,23
2020-11-29 23:51:00,0.0,0.0,0.00,0,0,0.000000,0,151.88,MH21,2020-11,23


In [11]:
#daily logs

car = []
volt = []
hrs = []
met = []
date_ = []
consum = []

grouped = total.groupby('meter')

for name, group in grouped:
    sub = grouped.get_group(name)
    date_list = list(np.unique(sub.index.date))
    
    for date in date_list:
        date_.append(date)
        met.append(name)
        s = sub.loc[sub.index.date==date]
        
        # daily consumption in kWh
        
        
        
        ac_on = s.loc[s['ac_on']==1]
        hrs.append((ac_on.shape[0]*3)/60)
        
        energy = ac_on['ac_power']/20000
        
        total_energy = energy.sum()
        
        c_on = s.loc[s['final_compressor']==1]
        if ac_on.shape[0]>0:
            car.append((c_on.shape[0]/ac_on.shape[0])*100)
        else:
            car.append(np.nan)

        volt.append(c_on['voltage'].mean())
        consum.append(total_energy)  

In [ ]:
da

In [13]:
daily_logs = pd.DataFrame({'Meter':met, 'Date':date_, 'ac_usage_hrs':hrs,'Avg CAR':car, 'Avg voltage when compressor on':volt, 'consumption':consum})

In [14]:
daily_logs.to_csv('FINAL/daily/'+'Daily logs_24meters_with_consumption.csv')

In [15]:
daily_logs

,Meter,Date,ac_usage_hrs,Avg CAR,Avg voltage when compressor on,consumption
0,BR06,2019-07-11,1.05,100.000000,256.898571,1.669118
1,BR06,2019-07-12,1.10,100.000000,248.661818,0.591609
2,BR06,2019-07-13,2.20,100.000000,249.681818,2.571931
3,BR06,2019-07-14,2.55,90.196078,238.495000,2.057216
4,BR06,2019-07-15,0.90,88.888889,249.434375,0.546768
...,...,...,...,...,...,...
8321,MH45,2020-11-25,0.00,NaN,NaN,0.000000
8322,MH45,2020-11-26,0.70,100.000000,244.282857,1.020138
8323,MH45,2020-11-27,0.00,NaN,NaN,0.000000
8324,MH45,2020-11-28,0.00,NaN,NaN,0.000000


In [27]:
t= daily_logs.groupby('Meter').mean()

In [28]:
t.to_csv('FINAL/final1/clustering/aug_10-6pm_avg_ac_hrs.csv')

In [21]:
daily_logs.to_csv()

,Meter,Date,ac_usage_hrs,Avg CAR,Avg voltage when compressor on
0,BR06,2019-07-11,1.05,100.000000,256.898571
1,BR06,2019-07-12,1.10,100.000000,248.661818
2,BR06,2019-07-13,2.20,100.000000,249.681818
3,BR06,2019-07-14,2.55,90.196078,238.495000
4,BR06,2019-07-15,0.90,88.888889,249.434375
...,...,...,...,...,...
9372,MH45,2020-11-25,0.00,NaN,NaN
9373,MH45,2020-11-26,0.70,100.000000,244.282857
9374,MH45,2020-11-27,0.00,NaN,NaN
9375,MH45,2020-11-28,0.00,NaN,NaN


In [3]:
# to be given to Ayushi

csv = pd.read_csv('FINAL/daily/daily_logs1.csv')

In [5]:
csv = csv.drop(['Unnamed: 0'],axis=1)

In [10]:
csv['Date'] = pd.to_datetime(csv['Date'],format = '%Y-%m-%d')

In [14]:
csv = csv.loc[~csv['Meter'].isin(['BR11','BR22','BR51'])]

In [25]:
bar = csv.loc[csv['Meter'].str.contains('B')]

In [26]:
bar = bar.loc[(bar['Date']<='2019-10-31') & (bar['Date']>='2019-07-01')]

In [28]:
df = pd.concat([mathura,bar],axis=0)

In [34]:
b = bar.groupby('Date').mean()

In [30]:
df.to_csv(output_folder+'Daily Usage Hours.csv')

In [42]:
o.to_csv(output_folder+'Average Daily Usage Hours overall jul-oct2019.csv')

In [40]:
o = df.groupby('Date').mean()

In [38]:
df = df.loc[(df['Date']>='2019-07-01')]

In [41]:
o

,ac_usage_hrs,Avg CAR,Avg voltage when compressor on
Date,,,
2019-07-01,9.442857,94.979613,233.501355
2019-07-02,10.228571,95.541018,227.734219
2019-07-03,11.123077,97.467949,229.565689
2019-07-04,9.125000,97.470404,228.899718
2019-07-05,6.366667,91.858823,224.198905
...,...,...,...
2019-10-27,0.908333,92.153680,250.258462
2019-10-28,0.891304,88.523502,253.032099
2019-10-29,1.060000,90.152801,250.508163


In [53]:
h = total.groupby(['meter',total.index.date,total.index.hour]).sum()

In [55]:
h['ac_on'] = (h['ac_on']*3)/60

In [56]:
h = h['ac_on']

In [60]:
h.plot()

TypeError: Data frame index is a pandas MultiIndex. pandas MultiIndex is not supported by plotly express at the moment.

In [62]:
h.to_csv('FINAL/final1/clustering/hourly_ac_use_hrs_data.csv')

In [65]:
total.groupby('meter').sum().shape[0]

24